In [1]:
import pandas as pd
import numpy as np
from sklearn import svm, neighbors,tree, ensemble, model_selection, preprocessing
from sklearn.model_selection import train_test_split


In [2]:
df = pd.read_csv('fulldata_colonnes_essentielles.csv')

In [3]:
# Les lignes ayant le même 'Accession' doivent être comprises comme les données d'une même entité (code Amal)
np.random.seed = 123

unique_users = df['ACCESSION'].unique()
target_n = int(.8 * len(df))
shuffled_users = np.random.permutation(unique_users)

user_count = df['ACCESSION'].value_counts()

mapping = user_count.reindex(shuffled_users).cumsum() <= target_n
mask = df['ACCESSION'].map(mapping)

df_train = df[mask]
df_test = df[~mask]

df_train.head(4)

,Unnamed: 0,ACCESSION,class1,class2,class3,AC_mode,ion_precursor_m/z,num_peak,peak_m/z,peak_rel.int.
0,7,PM000102,Phenylpropanoid,Phenylpropanoid glycoside,Phenylpropanoid glycoside,0,316,8,299.0,3
1,8,PM000102,Phenylpropanoid,Phenylpropanoid glycoside,Phenylpropanoid glycoside,0,316,8,180.0,10
2,9,PM000102,Phenylpropanoid,Phenylpropanoid glycoside,Phenylpropanoid glycoside,0,316,8,163.0,100
3,10,PM000102,Phenylpropanoid,Phenylpropanoid glycoside,Phenylpropanoid glycoside,0,316,8,145.0,60


In [4]:
#Séparation en variables cibles et explicatives :  chaque classe est une variable cible différente.
#La classe la plus interessante a priori est la classe 3, la plus 'précise' (classe 3 incluse dans la 2 qui est incluse dans la 1)
 
X_test = df_test[['num_peak', 'peak_m/z', 'peak_rel.int.', 'AC_mode' ]]
y_test3 = df_test.class3
y_test2 = df_test.class2
y_test1 = df_test.class1

X_train = df_train[['num_peak', 'peak_m/z', 'peak_rel.int.', 'AC_mode']]
y_train3 = df_train.class3
y_train2 = df_train.class2
y_train1 = df_train.class1

In [5]:
#classification paramètre par défaut rf CLASS3
clf_rf = ensemble.RandomForestClassifier()
clf_rf.fit(X_train, y_train3)

clf_rf.predict(X_test)
print('Score de rf:', clf_rf.score(X_test, y_test3))

Score de rf: 0.23655244029075806


In [6]:
#classification paramètre par défaut rf CLASS2
clf_rf = ensemble.RandomForestClassifier()
clf_rf.fit(X_train, y_train2)

clf_rf.predict(X_test)
print('Score de rf:', clf_rf.score(X_test, y_test2))

Score de rf: 0.32429906542056075


In [8]:
#classification paramètre par défaut rf CLASS1
clf_rf = ensemble.RandomForestClassifier()
clf_rf.fit(X_train, y_train1)

clf_rf.predict(X_test)
print('Score de rf:', clf_rf.score(X_test, y_test1))

Score de rf: 0.4814122533748702


In [7]:
#recherche des meilleurs paramètres forêt aléatoire CLASS3

clf_rf = ensemble.RandomForestClassifier()

parametres_rf = {}

grid_clf_rf = model_selection.GridSearchCV(estimator = clf_rf, param_grid = parametres_rf)

grille = grid_clf_rf.fit(X_train, y_train3)

print('Meilleurs paramètres de rf:', grid_clf_rf.best_params_)

y_pred_rf = grid_clf_rf.predict(X_test)

print('Score de rf:', grid_clf_rf.score(X_test, y_test3))